# 画像スクレイピング Notebook (ChromeDriver自動管理版)

このNotebookは、**Selenium**と**webdriver-manager**を利用します。webdriver-managerが、お使いのChromeブラウザに合った`chromedriver`を**自動でダウンロード・管理**してくれるため、手動での設定は不要です。

### ■ 使い方
1. **【重要】事前準備:** 必要なライブラリをインストールします。ターミナルで以下の2つのコマンドを実行してください。
    ```
    pip install selenium
    pip install webdriver-manager
    ```

2. 下の「設定」セルで、`TARGET_URL`や`SAVE_DIR`などの変数を、ご自身の環境に合わせて編集してください。
3. **「設定」セルを実行**します。（セルを選択して `Shift + Enter`）
4. **「スクレイピング実行」セルを実行**します。初回実行時に、適切な`chromedriver`が自動でダウンロードされ、その後ブラウザが起動します。

In [ ]:
import os

# --- 設定 (このセルを直接編集してください) ---

# 1. 画像を取得したいWebページのURL
TARGET_URL = "https://www.pixiv.net/artworks/100000000" # 実際のURLに変更してください

# 2. 画像を保存するディレクトリのフルパス
# (注意: このディレクトリは事前に作成しておく必要があります)
SAVE_DIR = "D:\\steganography\\manga"  # 例: "C:\\Users\\YourUser\\Downloads\\scraped_images"

# 3. 保存するファイル形式 (png, jpg, jpeg のいずれか)
SAVE_FORMAT = "jpg"

# 4. 自動スクロール設定
SCROLL_PAUSE_TIME = 2  # 各スクロール後に新しい画像が読み込まれるのを待つ秒数

# --- 設定はここまで ---

# ▼ 設定内容の確認とディレクトリの存在チェック
if not os.path.isdir(SAVE_DIR):
    print(f"エラー: 保存先ディレクトリ '{SAVE_DIR}' が存在しません。")
    print("上記の 'SAVE_DIR' を有効なパスに修正するか、ディレクトリを事前に作成してください。")
else:
    print("設定が読み込まれました。次のセルを実行してスクレイピングを開始してください。")
    print(f"  対象URL: {TARGET_URL}")
    print(f"  保存先: {SAVE_DIR}")
    print(f"  保存形式: .{SAVE_FORMAT}")

In [ ]:
# --- スクレイピング実行 ---

import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # ★★★ この行を追加 ★★★

# 設定セルでエラーが出ていないか再度チェック
if not os.path.isdir(SAVE_DIR):
    print(f"エラー: 保存先ディレクトリ '{SAVE_DIR}' が見つかりません。上の設定セルを修正・実行してください。")
else:
    # ★★★ webdriver-managerを使って、適切なChromeDriverを自動でインストール・セットアップ ★★★
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    try:
        # 1. ページにアクセス
        print(f"\nブラウザを起動し、'{TARGET_URL}' にアクセスします...")
        driver.get(TARGET_URL)
        time.sleep(5) # ページ初期読み込みの待機

        # 2. ページを最後までスクロールして、遅延読み込み画像をすべて表示させる
        print("ページをスクロールして、すべての画像を読み込んでいます...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        print("スクロール完了。")

        # 3. JavaScript実行後のHTMLを取得してBeautifulSoupで解析
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        img_tags = soup.find_all("img")

        if not img_tags:
            print("ページ内に画像が見つかりませんでした。")
        else:
            print(f"{len(img_tags)} 個の画像タグが見つかりました。ダウンロードを開始します。")
            # 4. 画像URLごとに処理
            saved_count = 0
            for i, img in enumerate(img_tags, start=1):
                img_url = img.get("src")
                if not img_url or not img_url.startswith('http'):
                    continue

                try:
                    print(f"({saved_count + 1}/{len(img_tags)}) ダウンロード中: {img_url[:80]}...")
                    img_data = requests.get(img_url, timeout=15, headers={'Referer': TARGET_URL}).content
                    
                    base_name, _ = os.path.splitext(os.path.basename(img_url).split('?')[0])
                    if not base_name:
                        base_name = f"image_{saved_count + 1}"
                    
                    filename = f"{saved_count + 1:03d}_{base_name}.{SAVE_FORMAT}"
                    save_path = os.path.join(SAVE_DIR, filename)

                    with open(save_path, "wb") as f:
                        f.write(img_data)
                    saved_count += 1
                    time.sleep(1)

                except requests.exceptions.RequestException as e:
                    print(f"  -> エラー: ダウンロードに失敗しました: {e}")
                except Exception as e:
                    print(f"  -> エラー: 保存中に予期せぬエラーが発生しました: {e}")

            print(f"\n完了しました。{saved_count}個の画像を'{SAVE_DIR}'に保存しました。")

    finally:
        if 'driver' in locals():
            driver.quit()
            print("ブラウザを終了しました。")